In [91]:
import json

# Years to load
years = [1, 501, 1000]
loaded_data = {}  # Store data in a dictionary with years as keys

for year in years:
    filename = f"network_year_{year}.json"
    
    try:
        with open(filename, "r") as f:
            loaded_data[year] = json.load(f)  # Load into dictionary
    
    except FileNotFoundError:
        print(f"⚠️ File {filename} not found! Skipping...")
    except json.JSONDecodeError:
        print(f"⚠️ Invalid JSON in {filename}! Skipping...")

# # Print loaded data
# for year, data in loaded_data.items():
#     print(f"\nYear {year}:")
#     print("Network Relations:", data["network_relation"])

In [92]:
# Re-import necessary modules after code execution environment reset
import networkx as nx
import json

num = 1000

json_file_path_1 = f"network_year_{num}.json"
with open(json_file_path_1) as f:
    data_year_1 = json.load(f)

# Extract connectivity information for Year 1
network_data_1 = data_year_1['network_relation']

# Build the network graph for Year 1
G_1 = nx.Graph()

# Add edges based on connectivity weights (non-zero only)
for source, details in network_data_1.items():
    source_id = int(source)
    for target, weight in details['connectivity'].items():
        target_id = int(target)
        if weight > 0:
            G_1.add_edge(source_id, target_id, weight=weight)

# Convert to GEXF for Gephi
gexf_output_path_1 = f"household_network_{num}.gexf"
nx.write_gexf(G_1, gexf_output_path_1)

In [93]:
import numpy as np

for year, year_data in loaded_data.items():
    network_relation = year_data['network_relation']
    
    # Step 1: Compute wealth per member
    wealth_per_member_list = []
    for group_id, group_data in network_relation.items():
        wpm = group_data['wealth'] / group_data['num_member']
        group_data['wealth_per_member'] = wpm
        wealth_per_member_list.append(wpm)

    # Step 2: Calculate quartile breakpoints (for 4 classes)
    quantiles = np.percentile(wealth_per_member_list, [25, 50, 75])  # 3 breakpoints for 4 classes
    
    # Step 3: Assign classes (1=richest, 4=poorest)
    for group_id, group_data in network_relation.items():
        wpm = group_data['wealth_per_member']
        if wpm <= quantiles[0]:
            wealth_class = 4
        elif wpm <= quantiles[1]:
            wealth_class = 3
        elif wpm <= quantiles[2]:
            wealth_class = 2
        else:
            wealth_class = 1

        group_data['wealth_class'] = wealth_class

# Verify the output
print("Wealth Distribution Breakpoints (4 classes):")
print(f"Class 4 (Poorest): ≤ {quantiles[0]:.2f}")
print(f"Class 3: ≤ {quantiles[1]:.2f}")
print(f"Class 2: ≤ {quantiles[2]:.2f}") 
print(f"Class 1 (Richest): > {quantiles[2]:.2f}")

from pprint import pprint
pprint(loaded_data)

Wealth Distribution Breakpoints (4 classes):
Class 4 (Poorest): ≤ 1.64
Class 3: ≤ 3.10
Class 2: ≤ 5.22
Class 1 (Richest): > 5.22
{1: {'Year': 1,
     'network_relation': {'1': {'connectivity': {'11': 0.5,
                                                 '14': 0.125,
                                                 '18': 0.2,
                                                 '2': 0.125,
                                                 '20': 0.3333333333333333,
                                                 '21': 0.3333333333333333,
                                                 '22': 0.3333333333333333,
                                                 '23': 0.3333333333333333,
                                                 '24': 0.16666666666666666,
                                                 '27': 0.1,
                                                 '29': 0.1111111111111111,
                                                 '3': 0.25,
                                         

In [94]:
import plotly.graph_objects as go
import numpy as np
from collections import defaultdict

# --- Configuration ---
year = 1000
relation = loaded_data[year]['network_relation']
num_classes = 4  # Set this to match your actual number of classes

# --- Class Naming System ---
class_names = {
    1: "Elite",
    2: "Upper",
    3: "Middle",
    4: "Lower"
}

# --- Step 1: Aggregate connectivity by wealth class ---
class_links = defaultdict(float)
for group_id, data in relation.items():
    from_cls = data['wealth_class']
    for target_id, weight in data.get('connectivity', {}).items():
        if target_id in relation:
            to_cls = relation[target_id]['wealth_class']
            class_links[(from_cls, to_cls)] += weight

# --- Step 2: Define nodes ---
left_labels = [f"{class_names[cls]} (Source)" for cls in sorted(class_names)]
right_labels = [f"{class_names[cls]} (Target)" for cls in sorted(class_names)]
all_labels = left_labels + right_labels
label_to_index = {label: i for i, label in enumerate(all_labels)}

# Node positioning
node_y_positions = np.linspace(0, 1, num_classes).tolist()
node_y = node_y_positions + node_y_positions  # Same for left/right
node_x = [0.0] * num_classes + [1.0] * num_classes  # Left/right columns

# Color scheme matching class tiers
class_colors = {
    1: "#2ca02c",  # Green for Elite
    2: "#1f77b4",  # Blue for Upper
    3: "#ff7f0e",  # Orange for Middle
    4: "#d62728"   # Red for Lower
}
node_colors = [class_colors[cls] for cls in sorted(class_names)] * 2

# --- Step 3: Process connections ---
all_weights = list(class_links.values())
min_conn = min(all_weights) if all_weights else 0
max_conn = max(all_weights) if all_weights else 1

source, target, value, link_colors, hover_text = [], [], [], [], []

for (from_cls, to_cls), conn in class_links.items():
    # Normalize and scale connection strength
    norm = (conn - min_conn) / (max_conn - min_conn) if max_conn != min_conn else 1
    scaled_value = norm**1.5  # Emphasize strong connections
    
    # Get visual properties
    source_idx = label_to_index[f"{class_names[from_cls]} (Source)"]
    target_idx = label_to_index[f"{class_names[to_cls]} (Target)"]
    base_color = class_colors[from_cls]
    
    # Convert hex to rgba with dynamic opacity
    r, g, b = [int(base_color[i:i+2], 16) for i in (1, 3, 5)]
    opacity = 0.1 + 0.9 * scaled_value  # Range: 10-100% opacity
    
    # Store properties
    source.append(source_idx)
    target.append(target_idx)
    value.append(scaled_value)
    link_colors.append(f"rgba({r}, {g}, {b}, {opacity})")
    
    hover_text.append(
        f"<b>{class_names[from_cls]} → {class_names[to_cls]}</b><br>"
        f"Raw strength: {conn:.2f}<br>"
        f"Relative strength: {scaled_value:.2f}"
    )

# --- Step 4: Create Visualization ---
fig = go.Figure(data=[go.Sankey(
    arrangement='snap',
    node=dict(
        pad=30,
        thickness=25,
        line=dict(color="black", width=0.8),
        label=all_labels,
        color=node_colors,
        x=node_x,
        y=node_y,
        hovertemplate="<b>%{label}</b><extra></extra>"
    ),
    link=dict(
        source=source,
        target=target,
        value=value,
        color=link_colors,
        hovertemplate="%{customdata}<extra></extra>",
        customdata=hover_text
    )
)])

fig.update_layout(
    title_text=f"Economic Class Connectivity (Year {year})",
    title_font_size=18,
    font=dict(size=12),
    width=700,
    height=600,
    hoverlabel=dict(
        bgcolor="white",
        font_size=14,
        font_family="Arial"
    )
)

fig.show()

In [95]:
def save_plotly_figure(fig, base_filename="class_connectivity"):
    """Save figure in multiple formats with optimal settings"""
    
    # 1. HTML (interactive)
    fig.write_html(f"{base_filename}.html",
                 include_plotlyjs='cdn',
                 full_html=False)
    
    print(f"Saved to {base_filename}.html")

# Usage
# save_plotly_figure(fig, "sankey_1000")

In [96]:
import networkx as nx
from collections import defaultdict
import numpy as np
import pandas as pd

# --- Step 1: Create NetworkX Graph from one year ---
year = 1000
relation = loaded_data[year]['network_relation']

G = nx.DiGraph()  # directed graph

# Add nodes and edges
for group_id, data in relation.items():
    G.add_node(group_id, wealth_class=data['wealth_class'], num_member=data['num_member'])

    for target_id, weight in data.get('connectivity', {}).items():
        if target_id in relation:
            G.add_edge(group_id, target_id, weight=weight)

# --- Step 2: Compute Centrality Measures ---
degree_centrality = nx.degree_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G, weight='weight')
eigenvector_centrality = nx.eigenvector_centrality(G)

closeness_centrality = nx.closeness_centrality(G)

# --- Step 3: Aggregate by Wealth Class ---
class_metrics = defaultdict(list)

for node in G.nodes:
    cls = G.nodes[node]['wealth_class']
    class_metrics[cls].append({
        'degree': degree_centrality[node],
        'betweenness': betweenness_centrality[node],
        'eigenvector': eigenvector_centrality[node],
        'closeness': closeness_centrality[node]
    })

# --- Step 4: Compute Averages ---
summary = []
for cls, metrics in class_metrics.items():
    df = pd.DataFrame(metrics)
    summary.append({
        'wealth_class': cls,
        'avg_degree': df['degree'].mean(),
        'avg_betweenness': df['betweenness'].mean(),
        'avg_eigenvector': df['eigenvector'].mean(),
        'avg_closeness': df['closeness'].mean()
    })

# Display as DataFrame
summary_df = pd.DataFrame(summary).sort_values('wealth_class')
summary_df


,wealth_class,avg_degree,avg_betweenness,avg_eigenvector,avg_closeness
3,1,2.0,0.000000,0.112509,1.0
0,2,2.0,0.006467,0.112509,1.0
1,3,2.0,0.013753,0.112509,1.0
2,4,2.0,0.002078,0.112509,1.0


In [89]:
class_metrics

defaultdict(list,
            {2: [{'degree': 2.0,
               'betweenness': 0.0,
               'eigenvector': 0.17149858514250882,
               'closeness': 1.0},
              {'degree': 2.0,
               'betweenness': 0.0,
               'eigenvector': 0.17149858514250882,
               'closeness': 1.0},
              {'degree': 2.0,
               'betweenness': 0.0,
               'eigenvector': 0.17149858514250882,
               'closeness': 1.0},
              {'degree': 2.0,
               'betweenness': 0.0,
               'eigenvector': 0.17149858514250882,
               'closeness': 1.0},
              {'degree': 2.0,
               'betweenness': 0.0404040404040404,
               'eigenvector': 0.17149858514250882,
               'closeness': 1.0},
              {'degree': 2.0,
               'betweenness': 0.0,
               'eigenvector': 0.17149858514250882,
               'closeness': 1.0},
              {'degree': 2.0,
               'betweenness': 0.02